In [32]:
import os
import json
from datetime import datetime, timedelta
import pandas as pd

def edit_short_memory(agent_folder, edit_date, wakeup_time="07:00"):
    with open(os.path.join(agent_folder, "short_memory_cache.json")) as f:
        ori_short_memory = json.load(f)

    last_date = datetime.strptime(edit_date, "%m-%d-%Y") - timedelta(days=1)
    last_date_data = pd.read_csv(os.path.join(agent_folder, f"activity_hist/{last_date.strftime('%m-%d-%Y')}.csv"))
    start_time = f"{edit_date} 00:00"
    end_time = f"{edit_date} {wakeup_time}"

    ori_short_memory.keys()
    ori_short_memory["cur_date"] = edit_date
    ori_short_memory["cur_time"] = "00:00"
    ori_short_memory["cur_activity"] = "Sleep"

    ori_short_memory["schedule"] = {'0':{
        "start_time" : start_time,
        "end_time" : end_time,
        "event": "Sleep"
    }}
    ori_short_memory["cur_event"] = {
        "start_time" : start_time,
        "end_time" : end_time,
        "event": "Sleep"
    }
    ori_short_memory["cur_event_index"] = 0
    ori_short_memory["cur_decompose"] = [{
        'activity': 'Sleep',
        'start_time': start_time,
        'end_time': end_time
    }]
    ori_short_memory["cur_decompose_index"] = 0

    ori_short_memory["cur_location_list"] = [{
        'start_time': start_time,
        'end_time': end_time,
        'location': last_date_data['location'].iloc[-1],
        'longitude': last_date_data['longitude'].iloc[-1],
        'latitude': last_date_data['latitude'].iloc[-1]
    }]
    ori_short_memory["cur_location_list_index"] = 0

    ori_short_memory["cur_heartrate_list"] = [{
        'mean': last_date_data["heartrate"].tail(30).mean(),
        'std': last_date_data["heartrate"].tail(30).std(),
        'start_time': start_time,
        'end_time': end_time
    }]
    ori_short_memory["cur_heartrate_list_index"] = 0

    ori_short_memory["cur_chatbot_dict"] = {}

    ori_short_memory["cache"] = []


    with open(os.path.join(agent_folder, "short_memory_cache.json"), "w") as f:
        json.dump(ori_short_memory, f)

In [ ]:
edit_short_memory(
    agent_folder=".Users/ba20c6a3-4736-3840-9d73-3093ff6b3885/agents/1",
    edit_date="03-01-2024",
    wakeup_time="08:00"
)

In [61]:
import os
import json
import random

def inject_intervention(case_folder, agents_list, plan):
    agents_folder=os.path.join(case_folder, "agents")
    intervent_agents = list(filter(lambda x: (x not in agents_list) and ("." not in x), os.listdir(agents_folder)))

    for _a in intervent_agents:
        _long_mem_file = os.path.join(agents_folder, f"{_a}/long_memory_cache.json")
        with open(_long_mem_file, "r") as f:
            _long_mem = json.load(f)
        plan["weekdays"] = random.sample([0,1,2,3,4,5,6], 3)
        _long_mem["intervention"] = plan
        with open(_long_mem_file, "w") as f:
            json.dump(_long_mem, f)

In [62]:
intervent_plan = {
    "intro" : "Hi, <name>, this is Dr. <name> from the text message walking study. Thank you for participating in this study. For the next 6 weeks we will be in touch through text messaging in regards to your walking. If you have any questions feel free to call/text at this number. Enjoyyour pedometer! Have a great week! :)",
    "9:00" : "Good morning <name>! A good way to monitor your walking is to set a goal! Say to yourself, I am going to walk more than I did yesterday! Set a specific goal for today!\nGood morning <name>! Today is a great day for a walk! Have you tried walking with a friend or family member? Instead of using the elevator have you tried walking up the stairs?",
    "12:30": "Today is a great afternoon to walk! Remember to always keep yourself hydrated by drinking lots of water!\nThis is a great time to check your step count! How many steps have you taken? Do you think you can walk more? Have a friend walk with you!",
    "17:30" : "Hey <name> how was your day today? Hope you had a great time walking! Keep up the great work for tomorrow too!\nOther motivational words Walking will make you healthier and feel great too! It doesn’t hurt to sweat a little when you walk! That means you are working out! Burning thecalories!\nWalking will help ease the mind and soul. Keep up the great walking! Check your pedometer! You are doing great, <name>. Keep on finding opportunities to walk more! Take breaks during work, park further away in parking lots! Go to the mall and walk!"
}

In [64]:
inject_intervention(
    case_folder=".Users/ba20c6a3-4736-3840-9d73-3093ff6b3885",
    agents_list=[],
    plan=intervent_plan
)